# Stamp selling program - by defining functions
#### The program is decomposed in terms of functionality that is required to perform each task, satisfying requirements of the program. They are decomposed as -
#### Firstly, defined function to take inpput (post type, destination, weight, unit price
#### Secondly, defined function to obtain values required for unit price calculation
#### Then, a function for creating shopping cart is defined followed by functionalities such as viewing cart, amending items in cart, removing items in a cart and checking out
#### Finally, all these functions are integrated in a main function with additional functionality for prompt user experience

#### NOTE: all_cart is our shopping cart, where at least one or multiple list of items are added (basically 2D array)

In [1]:
# Importing pandas for reading, writing, cleaning and manipulating provided csv files
# Importing datetime to extract real time date and time (system date and time) 

import pandas as pd
import datetime

# defining input for post type, with exception in place if incorrect input is provided upto certain level

def input_post_type():
    
    # catching exception if invalid post type is entered
           
    while True:
        try:
            post_type = input('Enter Post Type: ')
            
            # if no type is entered, raise exception
            
            if not post_type.strip():
                raise TypeError
                
            # if numbers are entered, raise exception
            
            elif post_type.isdigit():
                raise ValueError
            
            # raising exception if other strings are passed except parcel or post
            
            if post_type.lower() in 'parcel' or 'letter':
                assert post_type.lower() == 'parcel' or post_type.lower() == 'letter'          
           
            break 
            
        except (TypeError, ValueError, AssertionError):
            print("Oops!  That was no valid Entry.  Try parcel or letter ...")
            
    return post_type.lower()

# defining input for destination type with exception in place for invalid entries such as if destination falls in a provided 
# destination list , if post type is parcel, which destination and zone it can send it to 

def input_destination(post_type):
    
    df = pd.read_csv("Countries and Zones.csv")  # reading csv files, uses pandas package
    
    # this line keeps column labels to same format, by repalcing unwanted formats in csv file
    
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    
    if post_type == 'letter':  # basically checks if post type is letter, whether destination belongs to csv file
            
        while True:           # raising exception for invalid country input
            try:
                dest = input('Enter Destination: ')

            # assigning check to create new dataframe if destination country in df,
            ## using computational data frame returns True if present, else returns False

                check = df[df.destination_country == dest.title()]
                check_true = check.any(axis = 0)

            # checking condition if equal to false, raise exception

                if check_true.loc['destination_country'] == False:
                    raise Exception
                break
            except:
                print("Oops! Seems like destination country not in a system, Try different or full country name ...")

        
        return dest
    
    else:    # used else since post_type would be only parcel, that is left
        
        # reading csv files for parcel type with weiht as row index
        
        csvp = pd.read_csv('Economy Parcel Price Guide_by Sea ($).csv', index_col = 'Weight')

        while True: # raising exception if destination input is invalid for parcel type
            try:
                dest = input('Enter Destination: ')
                
                # manipulating dataframe to retireve zone from csv file
                
                check_zone = df[df.destination_country == dest.title()]
                zone_series = check_zone[check_zone.destination_country == dest.title()].zones
                
                zone = str(zone_series.iloc[0])
               # print(zone)

                clean_csvp = csvp.filter(items=['Zone 4', 'Zone 6', 'Zone 7', 'Zone 8', 'Zone 9']) # filtering out empty values

                # checking if zone belongs to parcel type
                
                if zone not in clean_csvp.columns:
                    raise Exception
                break   

            except:
                print("Oops! Parcel can not be sent to this destination, Please check your destination ...")
                
        #print(dest)
        return dest
        
        
# defining function to validate weight range for letter and parcel type
        
def input_weight(post_type):
    
    if 'letter' in post_type:
        
        while True:
            try:
                weight = float(input('enter weight in gram:'))
                weight = weight / 1000
                
                if weight <0 or weight > 0.5: # provided range for letter type
                    raise Exception
                break
            except:
                print('Weight is outside range for Letter type' + '\n' + 'Range is between 0 to 500 g')
                
    elif 'parcel' in post_type:
        
        while True:
            try:
                weight = float(input('enter weight in kg: ')) 
                if weight < 2.5 or weight >20: # # provided range for parcel type
                    raise Exception
                break
            except:
                print('Weight is outside range for parcel type' + '\n' + 'Range is between 2.5 to 20 kg')
                
    return weight


# defining function to obtain zone from destination using Countries and Zones.csv

       
def get_zone(dest):
    
    # converted csv file to dataframe with Destination country as row index
    
    zone_df = pd.read_csv('Countries and Zones.csv', index_col ='Destination country')
    
    zone_dict = zone_df.to_dict() # converted to dictionary
    val = zone_dict.get('Zones')  # obtained value as Zones from dictionary key as country name
    out = val.get(dest.title())   
    zone = out
    
    return zone

# defining function to assign index of weight for letter type

def get_index_weight_letter(weight):
    
    if weight <= 0.05:
        index = 0
    elif weight <= 0.25:
        index = 1
    elif weight <= 0.5:
        index = 2
        
    return index

# defining function to assign index of weight for parcel type

def get_index_weight_parcel(weight):
    
    if weight <= 3:
        index = 0
    elif weight <= 5:
        index = 1
    elif weight <= 10:
        index = 2
    elif weight <= 15:
        index = 3
    elif weight <= 20:
        index = 4
        
    index_p = index
        
    return index_p    

    
# function that adds items as a list called cart

def add_items(post_type, dest, weight, unit_price):
    
    cart = []               # initialising empty cart by appending each items
    cart.append(post_type)
    cart.append(dest)
    cart.append(weight) 
    cart.append(unit_price)
    
    return cart
    #print(cart)
    
# defining function that adds each cart to shopping cart , all_cart has been initialised at main function 
# and shopping(all_cart) is a function that works together with main() to add items to all_cart 
    
def shopping(all_cart):
    
    # each function defined above for input validation has been called here to start shopping 
    
    post_type = input_post_type() # calling post_type function returns post type as post_type
    #print(post_type)
    
    dest = input_destination(post_type) # calling destination function, returns destinationas as dest
    #print(dest)

    weight = input_weight(post_type) # calling weight function, returns weight
    #print(weight)
    
    zone = get_zone(dest) # calling get_zone function, returns zone
    #print(zone)
    
    # this code below retrieves unit price from provided csv for particular items
    # how it works is- we obtain index for each weight range and column label, and using it to retireve unit price
    
    if post_type == 'letter': # for the case of letter, reads repective csv 
        
        csv_letter = pd.read_csv('Economy Air Letters Price Guide ($).csv')
        
        csv_letter.drop('Weight', 1, inplace = True) # dropping Weight to clean dataframe for use
        
        # replacing column label for proper use of datafram for further manipulation
        
        clean_csv = csv_letter.rename(columns={"Zone 1":"Zone1", "Zone 2,3 and 5":"Zone2 Zone3 Zone5", "Zone 4, 6, 7, 8 and 9" : "Zone4 Zone6 Zone7 Zone8 Zone9" })
        
        # obtaining index for letter type using pre defined function get_index_weight_letter(weight)
        
        index = get_index_weight_letter(weight)
        #print(index)
        
        #the code below retireves column label into a list called column_list as three items in a list witn string value
        
        column_list = clean_csv.columns.tolist()
        new_zone = zone.replace(' ', '') # cleaning format of zone obtained from above defined function such that we can-
                                         # - find it in column_list
        array_column =[] # assigned new list that holds each each zones
        
        for item in column_list:
            array_column.append(item.split()) # appending to new list by splitting each item (group of zone into list)
            
        for item in array_column: # to obtain index of item which is a zone 
            if new_zone in item:
                pos = array_column.index(item) # postion of zone obtained from input destination into array_column
                #print(pos)
                
        csv_t = clean_csv.transpose()  # used transpose funtion of dataframe to change column level(which is zones) to index
        cost = csv_t.iloc[pos].loc[index] # used  .loc to obtain cost price from corresponding row and column index
        unit_price = cost
        #print(unit_price)
        
    elif post_type == 'parcel': # for a case of parcel, its slighty different as zones are well organised in csv file
        
        # reads csv, drops weight as not required, and filtered out zones with no values
        
        csv_parcel = pd.read_csv('Economy Parcel Price Guide_by Sea ($).csv')
        csv_parcel.drop('Weight', 1, inplace = True)
        clean_csv_parcel = csv_parcel.filter(items=['Zone 4', 'Zone 6', 'Zone 7', 'Zone 8', 'Zone 9'])
        
        index_p = get_index_weight_parcel(weight) # calling a function to obtain index of weight range in resp csv
        #print(index_p)
        
        cost_p = clean_csv_parcel.loc[index_p].loc[zone] # using weight range and zone to retireive cost price
        unit_price = cost_p
        #print(unit_price)
        
        
    # now this adds each inputs and corresponding cost price to a list called cart, and everytime items are added, appends to all_cart
    
    cart = add_items(post_type, dest, weight, unit_price)
    
    # this block of code is to check duplicate entery , and prompt end user either to add item or reject 
    # And, append each cart to all_cart(which is a shopping cart)
    
    if cart in all_cart:           
        print('\nDuplicate Entry')
        ask_confirm = input('Do you want to add anyway (y/n): ')
        
        if ask_confirm == 'y':   
            all_cart.append(cart) # if duplicate and promted out 'y' , adds to all_cart
        elif ask_confirm == 'n': # if 'n', advise so
            print('Item not added')        
    else:
        all_cart.append(cart) # in the case where no duplicate entry, appends cart to all_cart
    

    return all_cart


# defining function to view shopping cart

def view_cart(all_cart):
    
    prompt = input('Do you want to view cart (y/n): ')
    
    # display items in a shopping cart to a suitable format
    
    if prompt == 'y':
        for i in range(0, len(all_cart)):
            formatted_str = 'item no: '+str(i+1) + ' item type: '+ str(all_cart[i][0])+' '+'destination: '+str(all_cart[i][1])+' '+ 'weight: '+str(all_cart[i][2])+' '+'kg'+' '+'unit price: '+str(all_cart[i][3])
            print(formatted_str)

            
#defining function to check for a cart in a case where items are updated or removed, display anyway if updated
        
def cart_check(all_cart):
    for i in range(0, len(all_cart)):
        formatted_str = 'item no: '+str(i+1) + ' item type: '+ str(all_cart[i][0])+' '+'destination: '+str(all_cart[i][1])+' '+ 'weight: '+str(all_cart[i][2])+' '+'kg'+' '+'unit price: '+str(all_cart[i][3])
        print(formatted_str)

        
# defining a function to amend item in shopping cart only by weight, when user provides valid item number
        
def amend_cart(all_cart):
    while True:     # raising exception if other than yes or no is a input
        try:
            prompt = input('Do you want to amend item: ')
            yes_list = ['y', 'Y', 'yes', 'Yes', 'YES', 'n', 'N', 'No', 'NO', 'no']
            if prompt not in yes_list:
                raise Exception
            break
        except: 
            print('enter: ' + str(yes_list))
            
    if prompt == 'y':
        
        prompt_again = int(input('Enter item number: '))
        
        post_type = all_cart[prompt_again-1][0] # to check for a post_type if letter or parcek in an item number
        
        new_weight = input_weight(post_type) # called input weight function with post_type identified from item number
        
        
        all_cart[prompt_again - 1][2] = new_weight # update weight
        
        cart_check(all_cart)
        
    elif prompt == 'n':
        pass
    
    return all_cart


# defining a funciton to remove item from shopping cart

def remove_item(all_cart):
    while True:   # raising exception
        try:
            prompt = input('Do you want to remove item: ')
            yes_list = ['y', 'Y', 'yes', 'Yes', 'YES', 'n', 'N', 'No', 'NO', 'no']
            if prompt not in yes_list:
                raise Exception
            break
        except: 
            print('enter: ' + str(yes_list))
            
    if prompt == 'y':
        prompt_again = int(input('Enter item number: '))
        
        del all_cart[prompt_again -1] # deleting item number identified from all_cart, using indexing
        
        cart_check(all_cart)
        
    elif prompt == 'n':
        pass
        
    return all_cart


# defining check out function, if items are empty, check out with no invoice printed and updating sales history csv
# if check out is prompted yes, prints invoice adn append each items to sles history csv
# if check out is promted no, ask user to continue shopping, if user input is no, then terminates shopping to avoid indefinite loop


def checkout(all_cart):
       
    x = 0
    sum = 0
    while x < len(all_cart):
        sum += all_cart[x][3]
        x += 1
        total_cost = sum  # calculates total cost
        
    if len(all_cart) == 0:
        
        print('Thank You for using out Post Service') # if shopping cart is empty terminates shopping
        
    else:
        
        print('Thank You for shopping, Have a great day ...') # terminates shopping with invoice printed
        
        # this code block is to print invioce into a specific format, thus formats are defined as below-

        formatted_str = '--------------------Invoice--------------------------\n\n\n'

        for i in range(0, len(all_cart)): 

            formatted_str += 'item no: '+str(i+1) + ' item type: '+ str(all_cart[i][0])+' '+'destination: '+str(all_cart[i][1])+' '+ 'weight: '+str(all_cart[i][2])+' '+'kg'+' '+'unit price: '+str(all_cart[i][3])+'\n\n'

        formatted_str += '\n\n\nTotal Cost: $'+' '+ str(total_cost)+'\n'   
        formatted_str +='----------------End Invoice-----------\n\n\n\n'

        for i in range(0, len(all_cart)):
            formatted_str +='------------Purchased Stamps-----------------------\n'
            formatted_str += str(all_cart[i][0]) +'\n'+'Destination: '+str(all_cart[i][1])+'               '+'Weight: '+ str(all_cart[i][2])+' '+'kg\n'
            formatted_str +='-----------------------------------------------\n\n\n'


        # this code block is to format date and time    
            
        file_date_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M")
        file_name = str(file_date_time)

        current_date_time = datetime.datetime.now().strftime("%d/%m/%y %H:%M") # imported date and time function
        strdatetime = str(current_date_time)
        
        # this code block is to generate '.txt' file as invoice, unique to date and time

        with open(file_name +'.txt', 'w') as file_handle:
            file_handle.write(formatted_str) 
            
        # this code block is to append each items to a sales history, identified by same number in increasing order for single transaction

        with open('sales_history.csv', 'r') as file_handle: # splits line, identifies last line and firt item to generate count
            lines = file_handle.read().splitlines()
            last_line =lines[-1]
            count = int(last_line.split(',')[0]) # here count is incremented

        with open('sales_history.csv', 'a') as file_handle: # append items to sales history
            for item in all_cart:
                file_handle.write(str(count+1)+','+strdatetime +','+str(item[0])+','+str(item[2])+','+str(item[1])+','+str(item[3]))
                file_handle.write('\n')

                
# this function defines list of activities which are for viewing shopping cart, amending shopping cart and remvoing shoppign cart

    
def menu(all_cart):
    
   # calling all such functions defined earlier

    view_cart(all_cart)
        
    amend_cart(all_cart)
    #view_cart(all_cart)
    
    remove_item(all_cart)
    #view_cart(all_cart)
    
# this is a main function definition, where empty all_cart is defined as a shopping cart 

def main():
    
    all_cart = []
    
    # this block of code is to keep a loop of adding items to shopping cart as a single transaction until user does not want to continue shopping
   
    while True:
            
        ask = input('Do you want to continue shopping (y/n): ')
        
        if ask == 'y':
            all_cart = shopping(all_cart)
        else:
            break      
    # here menu is called upon to perform activites
    
    menu(all_cart)
    
     # this block of code is to prompt out for check out, if yes, perform such operation identified above in defining checkout function
    # if promts 'no', ask user again to continue ro not, if yes performs all operation as usual if no, terminates transaction
    
    while True:
        prompt = input('Do you want to checkout (y/n): ')
        if prompt == 'y':
            checkout(all_cart)
            break
        else:
            ask = input('Do you want to continue shopping (y/n): ')
            if ask == 'y':
                all_cart = shopping(all_cart)
                menu(all_cart)
                
            else:
                print('\n\nThank you for using our post system')
                break    
    
    
    
if __name__ == "__main__":
    main()   
   
  # Thank You.    
    


Do you want to continue shopping (y/n): n
Do you want to view cart (y/n): n
Do you want to amend item: n
Do you want to remove item: n
Do you want to checkout (y/n): n
Do you want to continue shopping (y/n): y
Enter Post Type: parcel
Enter Destination: Brasil
Oops! Parcel can not be sent to this destination, Please check your destination ...
Enter Destination: post
Oops! Parcel can not be sent to this destination, Please check your destination ...
Enter Destination: Brasil
Oops! Parcel can not be sent to this destination, Please check your destination ...
Enter Destination: US
Oops! Parcel can not be sent to this destination, Please check your destination ...
Enter Destination: United States
enter weight in kg: 3
Do you want to view cart (y/n): y
item no: 1 item type: parcel destination: United States weight: 3.0 kg unit price: 42.79
Do you want to amend item: n
Do you want to remove item: n
Do you want to checkout (y/n): y
Thank You for shopping, Have a great day ...
